In [1]:
# %pip install langchain nltk openai pandas 

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----- ---------------------------------- 0.2/1.5 MB 3.1 MB/s eta 0:00:01
     -------------------- ------------------- 0.8/1.5 MB 6.1 MB/s eta 0:00:01
     -------------------------------- ------- 1.2/1.5 MB 7.8 MB/s eta 0:00:01
     ---------------------------------------  1.5/1.5 MB 8.0 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 6.9 MB/s eta 0:00:00
  Using cached pandas-2.1.4-cp311-cp311-win_amd64.whl.metadata (18 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ---------------------------------------- 65.6/65.6 kB ? eta 0:00:00
  Using cached requests-2.31

In [3]:
# ovveride default sqlite3 with newer version if needed (cause of some chromadb bug)
# %pip install pysqlite3-binary
# __import__("pysqlite3")
# import sys
# sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

import csv
import os
import langchain as lc
import nltk
import openai
import pandas as pd
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

from data_collecting.wikitravel_scraper import WikiTravelScraper

sample data

In [2]:
data = {"Herat":"You can do swimming here. You can play sports. Smoking  is allowed.",
        "Bangalore":"Bangles, sports, sushi",
        "Mangalore":"Mangoes, sports, sushi",
        "Paris":"Eiffel tower, sports, jet skiing",
        "New York":"Sports,smoking, skiing"}

In [4]:
docs = [] # here each doc would be a wikitravel info page
for place_name,place_info in data.items():
    to_metadata = {"place_name": place_name}
    # values_to_embed = {"place_info": place_info, "place_name": place_name,"weather":"cold"} etc
    string_to_embed = place_info#"\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
    newDoc = Document(page_content=string_to_embed, metadata=to_metadata)
    docs.append(newDoc)

In [9]:
splitter = CharacterTextSplitter(separator = "\n",
                                chunk_size=500, 
                                chunk_overlap=0,
                                length_function=len)
documents = splitter.split_documents(docs)
# Now that we have the chunks, we will generate the embeddings and insert the values into Chroma. 
# Each vector inserted will have both the vector representation that will be used for similarity search as well as the metadata values we added.

# Generate embeddings from documents and store in a vector database
embeddings_model = OpenAIEmbeddings()
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [10]:
query = "water"
docs = db.similarity_search(query)
docs

[Document(page_content='You can do swimming here. You can play sports. Smoking  is allowed.', metadata={'place_name': 'Herat'}),
 Document(page_content='Sports,smoking, skiing', metadata={'place_name': 'New York'}),
 Document(page_content='Eiffel tower, sports, jet skiing', metadata={'place_name': 'Paris'}),
 Document(page_content='Mangoes, sports, sushi', metadata={'place_name': 'Mangalore'})]